In [2]:
# 데이터 프레임 
import pandas as pd 
import numpy as np

# 이상치제거 모델 
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor 
from random import seed


# 스케일링 
from sklearn.preprocessing import MinMaxScaler 


# 변수 그룹 별 전처리 
from sklearn.compose import make_column_transformer


# 파이프라인 
from sklearn.pipeline import Pipeline

# 훈련, 테스트 데이터 준비 
from sklearn.model_selection import train_test_split

# 특성 추출 모델 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso 
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet

- 데이터의 분포가 극단적으로 모였을 때 (포아송 분포)
- 선형 모델은 데이터가 정규분포일 때 적합 

In [12]:
df_bi = pd.read_csv("./data/train_bicycle.csv")
df_bi.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [13]:
# 시간 추출 
def hour_split(x) : 
    result = x.split()
    return result[1]

df_bi['hour'] =  df_bi['datetime'].apply(hour_split)
df_bi['hour'].unique()

array(['00:00:00', '01:00:00', '02:00:00', '03:00:00', '04:00:00',
       '05:00:00', '06:00:00', '07:00:00', '08:00:00', '09:00:00',
       '10:00:00', '11:00:00', '12:00:00', '13:00:00', '14:00:00',
       '15:00:00', '16:00:00', '17:00:00', '18:00:00', '19:00:00',
       '20:00:00', '21:00:00', '22:00:00', '23:00:00'], dtype=object)

In [14]:
# 연도 추출
def year_split(x) : 
    result = x.split()
    return result[0][:4]
df_bi['year'] = df_bi['datetime'].apply(year_split)
df_bi['year'].unique()

array(['2011', '2012'], dtype=object)

In [15]:
# 월 추출 
def month_split(x) : 
    result = x.split()
    return result[0][5:7]
df_bi['month'] = df_bi['datetime'].apply(month_split)
df_bi['month'].unique()

# 일자는 사용 안 함 

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12'], dtype=object)

In [16]:
# datetime 제거 
df_bi.drop('datetime', axis = 1, inplace= True)
df_bi.head(2)

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,hour,year,month
0,1,0,0,1,9.84,14.395,81,0.0,3,13,16,00:00:00,2011,01
1,1,0,0,1,9.02,13.635,80,0.0,8,32,40,01:00:00,2011,01


In [17]:
# 원핫인코딩 
print("전 : ",df_bi.shape)
df_bi = pd.get_dummies(df_bi, columns= ['season', 'weather','hour', 'year', 'month'])
print("후 : ", df_bi.shape)
df_bi.head()

전 :  (10886, 14)
후 :  (10886, 55)


,holiday,workingday,temp,atemp,humidity,windspeed,casual,registered,count,season_1,...,month_03,month_04,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12
0,0,0,9.84,14.395,81,0.0,3,13,16,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,9.02,13.635,80,0.0,8,32,40,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,9.02,13.635,80,0.0,5,27,32,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,9.84,14.395,75,0.0,3,10,13,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,9.84,14.395,75,0.0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# 이상치 확인 
import warnings 
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt 
import seaborn as sns 

In [19]:
li_col = ['temp', 'atemp', 'humidity',
       'windspeed']

# 이상치 제거 
def get_outlier(df, col) : 
    Q25= np.percentile(df[col].values, 25)
    Q75= np.percentile(df[col].values, 75)
    
    IQR = Q75-Q25 
    
    lowest = Q25 - 1.5*IQR
    highest = Q75 + 1.5*IQR 
    
    outlier_idx = df[col][(df[col] < lowest) | (df[col] > highest)].index 
    
    return outlier_idx 

for col in li_col : 
    outlier_idx = get_outlier(df_bi, col)
    df_bi.drop(outlier_idx, axis = 0, inplace= True)

In [20]:
df_bi.shape

(10638, 55)

In [21]:
# Scaler 
li_col = ['temp', 'atemp', 'humidity',
       'windspeed']
for col in li_col : 
    df_bi[col] = (df_bi[col] - min(df_bi[col])) / (max(df_bi[col]) - min(df_bi[col]))

df_bi.head()

,holiday,workingday,temp,atemp,humidity,windspeed,casual,registered,count,season_1,...,month_03,month_04,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12
0,0,0,0.224490,0.305068,0.793478,0.0,3,13,16,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0.204082,0.288064,0.782609,0.0,8,32,40,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0.204082,0.288064,0.782609,0.0,5,27,32,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0.224490,0.305068,0.728261,0.0,3,10,13,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0.224490,0.305068,0.728261,0.0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df_bi.describe()

,holiday,workingday,temp,atemp,humidity,windspeed,casual,registered,count,season_1,...,month_03,month_04,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12
count,10638.000000,10638.000000,10638.000000,10638.000000,10638.000000,10638.000000,10638.000000,10638.000000,10638.000000,10638.000000,...,10638.000000,10638.000000,10638.000000,10638.000000,10638.000000,10638.000000,10638.000000,10638.000000,10638.000000,10638.000000
mean,0.028671,0.680015,0.484261,0.513762,0.590622,0.396259,36.087610,155.795074,191.882685,0.239237,...,0.079432,0.081782,0.084884,0.084696,0.085072,0.084978,0.084508,0.084790,0.084884,0.085166
std,0.166887,0.466492,0.193961,0.189026,0.205445,0.240094,49.987538,151.139015,181.238751,0.426638,...,0.270425,0.274046,0.278723,0.278442,0.279002,0.278863,0.278162,0.278583,0.278723,0.279142
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.326531,0.355856,0.423913,0.225848,4.000000,36.000000,42.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.489796,0.525338,0.586957,0.354874,17.000000,118.000000,145.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,0.632653,0.677928,0.760870,0.548303,49.000000,223.000000,285.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,367.000000,886.000000,977.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Study 에서 알게 된 점
### feature selection 
- 분산으로 feature를 제거할 수 있다. 
- 카테고리가 많으면 원핫인코딩을 한 후에 분산을 확인해본다 

- lasso는 alpha 값이 작아질수록 feature 개수가 적어진다 

### 전진선택법, 후진제거법 
- SequentialFeatureSeletor 
- 이걸 사용할 때, 모델 기반으로 하면 된다. (내가 했던 것에서는 릿지 라쏘 같은 것을 사용하면 될 듯)

### 기타 
- 단변량 분석 시, 언제 F분포 사용? 언제 Chi2 사용? (기초통계 분포 공부 필요)

- y값에 log 씌워서 한다..? 
- 평가지표 종류 

- RMLSE : kaggle ~ y값에 log를 씌웠을 때 ~ 그러면 왜 y값에 log 씌우는가? 





In [23]:
from sklearn.model_selection import train_test_split
X = df_bi.drop(['count', 'casual', 'registered'], axis = 1)
y = df_bi['count']

x_train, x_test, y_train, y_test = train_test_split(X, y , random_state = 0)

In [29]:
from sklearn.feature_selection import SelectPercentile

select = SelectPercentile(percentile=70).fit(x_train, y_train) # 70%만 사용 

x_train_select = select.transform(x_train)
x_test_selected  = select.transform(x_test)

print(x_train.shape)
print(x_train_select.shape)

(7978, 52)
(7978, 36)


In [32]:
# 어떤 특성이 선택되었는가 
idx = select.get_support()
for index, value in enumerate(idx) : 
    if value : 
        print(x_train.columns[index], end = ' / ')

workingday / temp / atemp / humidity / windspeed / season_1 / season_2 / season_3 / season_4 / hour_00:00:00 / hour_01:00:00 / hour_02:00:00 / hour_03:00:00 / hour_04:00:00 / hour_05:00:00 / hour_08:00:00 / hour_09:00:00 / hour_11:00:00 / hour_12:00:00 / hour_13:00:00 / hour_14:00:00 / hour_15:00:00 / hour_16:00:00 / hour_17:00:00 / hour_18:00:00 / hour_19:00:00 / hour_20:00:00 / year_2011 / year_2012 / month_05 / month_06 / month_07 / month_08 / month_09 / month_10 / month_11 / 

In [30]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(x_train_select, y_train)

print("lr.coef_: {}".format(lr.coef_)) # 기울기 파라미터 (가중치 w)

print("lr.intercept_ : {}".format(lr.intercept_)) # 절편 (b)

print("훈련 세트의 정확도 : {:.2f}".format(lr.score(x_train_select,y_train)))

print("테스트 세트의 정확도 : {:.2f}".format(lr.score(x_test_selected,y_test)))

lr.coef_: [ 1.24466014e+00  5.99780022e+01  1.49728575e+02 -1.23849110e+02
 -1.85546662e+01  1.46513064e+14  1.46513064e+14  2.05912996e+14
  1.46513064e+14 -8.64994831e+01 -1.00800688e+02 -1.14865912e+02
 -1.22853546e+02 -1.24663761e+02 -1.07830382e+02  2.26562749e+02
  7.86662935e+01  4.67510119e+01  8.07443964e+01  7.99736717e+01
  5.75714310e+01  6.98103358e+01  1.30477611e+02  2.89741459e+02
  2.62739923e+02  1.53043595e+02  6.65301238e+01  6.39504138e+04
  6.40356917e+04  2.92038484e+01  1.66857797e+01 -5.93999321e+13
 -5.93999321e+13 -5.93999321e+13  1.71241235e+01  4.97320791e+00]
lr.intercept_ : -146513064022821.8
훈련 세트의 정확도 : 0.67
테스트 세트의 정확도 : 0.67


In [31]:
from sklearn.linear_model import Lasso 

lasso = Lasso().fit(x_train_select, y_train)

print("훈련 세트의 정확도 : {:.2f}".format(lasso.score(x_train_select, y_train)))

print("테스트 세트의 정확도 : {:.2f}".format(lasso.score(x_test_selected, y_test)))

print("사용한 특성의 수 : {}".format(np.sum(lasso.coef_ != 0)))

print("사용한 max_iter : {}".format(lasso.n_iter_))

훈련 세트의 정확도 : 0.65
테스트 세트의 정확도 : 0.65
사용한 특성의 수 : 28
사용한 max_iter : 177


In [27]:

lasso001 = Lasso(alpha=0.01, max_iter=100000).fit(x_train, y_train)

print("훈련 세트의 정확도 : {:.2f}".format(lasso001.score(x_train, y_train)))

print("테스트 세트의 정확도 : {:.2f}".format(lasso001.score(x_test, y_test)))

print("사용한 특성의 수 : {}".format(np.sum(lasso001.coef_ != 0)))

print("사용한 max_iter : {}".format(lasso001.n_iter_))

print()

lasso00001 = Lasso(alpha=0.0001, max_iter=100000).fit(x_train, y_train)

print("훈련 세트의 정확도 : {:.2f}".format(lasso001.score(x_train, y_train)))

print("테스트 세트의 정확도 : {:.2f}".format(lasso001.score(x_test, y_test)))

print("사용한 특성의 수 : {}".format(np.sum(lasso001.coef_ != 0)))

print("사용한 max_iter : {}".format(lasso001.n_iter_))

print()

훈련 세트의 정확도 : 0.70
테스트 세트의 정확도 : 0.69
사용한 특성의 수 : 48
사용한 max_iter : 417

훈련 세트의 정확도 : 0.70
테스트 세트의 정확도 : 0.69
사용한 특성의 수 : 48
사용한 max_iter : 417



In [28]:
from sklearn.linear_model import Ridge
ridge = Ridge().fit(x_train, y_train)

print("훈련 세트의 정확도 : {:.2f}".format(ridge.score(x_train,y_train)))

print("테스트 세트의 정확도 : {:.2f}".format(ridge.score(x_test,y_test)))

훈련 세트의 정확도 : 0.70
테스트 세트의 정확도 : 0.69


# 회귀 평가지표
실제 값과 회귀 예측값의 차이를 기반으로 함 

[참고1](https://bkshin.tistory.com/entry/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-17-%ED%9A%8C%EA%B7%80-%ED%8F%89%EA%B0%80-%EC%A7%80%ED%91%9C)
[참고2](https://velog.io/@tyhlife/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-%ED%9A%8C%EA%B7%80-%EB%AA%A8%EB%8D%B8%EC%9D%98-%ED%8F%89%EA%B0%80-%EC%A7%80%ED%91%9C)

In [3]:
import numpy as np 

y = np.array([67,78,91])
y_hat = np.array([60,80,90])

## MAE 
실제값과 예측 값의 차이를 절대값으로 변환해 평균한 것 

> 장점 
- 지표 자체가 직관적이며 예측변수와 단위가 같다. 
    - ex) 기온예측 모델의 MAE가 3이면, 평균적으로 3도 정도 잘못예측한다 

> 단점 
- 잔차에 절댓값을 씌우기 때문에 underestimates인지 over인지 알 수 없다. 
- 스케일에 의존적이다. (MAE, MSE, RMSE와 동일)
    - 예를 들어, 비트코인 가격이 25000이고, 이더리움 가겨이 600일 때 두 모델의 MAE가 모두 100이라면 `동일한 에러율이 아님에도 불구하고 MAE는 동일하다.`

In [4]:
n = 3 

mae = sum(abs(y-y_hat))/n
mae

3.3333333333333335

## MSE 
실제 값과 예측 값의 차이를 제곱해 평균한 것 

>장점 
- 지표자체가 직관적이다. 

>단점 
- 예측 변수와 단위가 다르다 
    - 기온을 예측하는 모델의 MSE가 4라면, 평균적으로 2도 정도 잘못 예측하는 것 
    - 스케일에 의존적이다. 

- 잔차를 제곱하기 때문에 `이상치에 민감`하다.
- 잔차를 제곱하기 때문에, 1미만의 에러는 더 작아지고, 그 이상은 더 커진다. 
- 실제값에 대해 underestimates인지 over인지 파악하기 힘들다


In [11]:
mse = sum(np.square(y-y_hat))/n
mse

18.0

## RMSE 
MSE 값은 오류의 제곱을 구하므로 실제 오류 평균보다 더 커지는 특성이 있어, MSE에 루트를 씌운 RMSE 값을 쓰는 것 

> 장점
- 지표 자체가 직관적이며 예측변수와 단위가 같다. 
    -기온예측 모델의 RMSE가 3이면, 평균적으로 3도 정도를 잘못 예측하는 것임 
- 잔차를 제곱하기 때문에 `이상치에 민감`하다.
- 제곱된 자차를 다시 루트로 풀어주기 때문에 잔차를 제곱해서 생기는 값의 왜곡이 MSE에 비해 좀 덜하다. 

> 단점 
- 실제 값에 대해 under인지 over인지 파악하기 힘들다. 
- 스케일에 의존적이다.

In [14]:
mse = sum(np.square(y-y_hat))/n
rmse = mse**(1/2)
rmse

4.242640687119285

## MAPE
MAE를 비율로 표현한 것 

> 장점 
- 지표자체가 직관적이다 
    - 공연석 예매 예측 모델의 MAPE가 3%인 경우, 실제 공연석의 예매량과 예측 예매량 비율이 3% 정도 차이난다고 할 수 있음
- 비율 변수이기 때문에 MAE, MSE, RMSE에 비해 비교가 용이하다 

> 단점 
- under 인지 over인지 알 수 없다
- 비율로의 해석이 의미있는 값에만 적용할 수 있음 
- 실제 값에 0이 포함된 경우  MAPE로 계산할 수 없다



## MSLE 
MSE에 로그를 적용해준 지표임 
log(y+1)이다. y=0일때, 마이너스 무한대인 것을 보정해주기 위해 +1을 해준 것 

In [15]:
msle = sum((np.log(y+1) - np.log(y_hat+1))**2)/n

msle

0.004181940368561632

## RMSLE 
RMSE에 로그를 적용해준 지표

> 장점이 뭐죠? 
 - RMSE보다 아웃라이어에 강해진다. 아웃라이어가 있더라도 RMSE는 굉장히 증가하지만, RMSLE는 증가폭이 미미하다 
 - 상대적 error를 측정해준다 
    - 로그 공식에 의해, 예측값과 실제값의 로그의 상대적 비율을 구할 수 있다. 
    - 절대적 크기가 변하더라도, 상대적 크기가 동일하면, `RMLSE값도 동일`하다. 하지만 RMSE는 변한다. 
 - `Under Estimation에 큰 패널티`를 부여한다 = 예측값이 실제값보다 작을 때, 더 높은 패널티가 주어진다. 
    - ex) 배달음식이 30분 걸리다고 했는데 실제로 20분 걸리는 건 ok, 근데 40분 걸리면 안됨 


In [17]:
rmsle = msle**(1/2)
rmsle

0.0646679237996832

## R-sqaure 

분산 기반으로 예측 성능을 평가 => 1에 가까울수록 예측 정확도가 높다 

예측값 분산 / 실제값 분산

In [19]:
np.var(y_hat)/np.var(y)

1.6166281755196306